In [1]:
import os
import pandas as pd
import time
from collections import Counter
import bz2
import json
import re

In [2]:
def process_files(data_dir, output_file_path, can_reuse_output = True):
    if os.path.isfile(output_file_path) and can_reuse_output:
        return
    
    filenames = [filename for filename in os.listdir(data_dir) if filename.endswith('.json.bz2')]
    input_files_paths = [os.path.join(data_dir, filename) for filename in filenames]

    domain_matcher = re.compile(r"^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?(?P<domain>[^:\/?\n]+)")
    get_domain_from_url = lambda string: domain_matcher.match(string).group('domain')
        
    with bz2.open(output_file_path, 'wb') as output_file:
        for input_file_path in input_files_paths:
            start = time.time()
            
            with bz2.open(input_file_path, 'rb') as input_file:
                for i, line in enumerate(input_file):
                    line = json.loads(line)
                    
                    data_line = {'quote_word_count': Counter(line['quotation']),
                                 'speaker'         : line['speaker'],
                                 'qids'            : line['qids'],
                                 'first_date'      : line['date'],
                                 'num_occurrences' : line['numOccurrences'],
                                 'domains'         : [get_domain_from_url(url) for url in line['urls']]}

                    output_file.write((json.dumps(data_line) + '\n').encode('utf-8'))
                    
                    if not i % 1000000:
                        print("Read", i, "lines from", input_file_path, 'in', (time.time() - start) / 60, "minutes")
                        
            print("Finished reading", input_file_path, 'in', (time.time() - start) / 60, "minutes")

def load_col_from_json(input_file_path, columns):
    """
    WARNING: DONT USE THIS UNLESS YOU WANT A BROKEN COMPUTER
    """
    columns_dict = {key: [] for key in columns}
    
    with bz2.open(input_file_path, 'rb') as input_file:
        start = time.time()
        
        for i, line in enumerate(input_file):
            line = json.loads(line)
            
            for column in columns:
                columns_dict[column].append(line[column])
                
            if not i % 1000000:
                print("Read", i, "lines from", input_file_path, 'in', (time.time() - start) / 60, "minutes")
                
    return pd.DataFrame(columns_dict)

In [3]:
DATA_DIR = 'Data'
CACHE_DIR = 'Cache'
CACHE_FILE_PATH = os.path.join(CACHE_DIR, 'processed_data.json.bz2')

process_files(DATA_DIR, CACHE_FILE_PATH)

# POUR KAOU

Bonjour Kaou, tu as choisi word counting. Malheureusement je me suis rendu trop tard dans l'execution de la cellule précédente que j'avais oublié d'enlever les punctuation. Oups. Tu vas devoir itérer sur tous les clés du Compteur et enlever manuellement, et merge aussi les mêmes mots. Ou juste relancer la cellule du haut que j'ai déjà corrigé, mais que je n'ai pas eu le courage de relancer parce que ça prend beaucoup trop de temps (mais pas de resources, donc ne te fais pas de soucis tu peux le run sur n'importe quoi).

Lis cette page pour des idées sur comment faire évoluer ton travail au délà du word counting: https://scikit-learn.org/stable/modules/feature_extraction.html

Je te mets aussi à disposition une simple fonction qui efface les "English stop words" d'une liste de mots. Par contre, fais gaffe à comment tu l'utilises. Les stop words ne sont pas nécéssairement toutes toujours inutiles. Lis sklearn pour les détails et affiche la liste avant d'utiliser la fonction pour voir si elle t'arrange. Tu peux covertir la fonction pour qu'elle droppe le compte des stop words dans un compteur en faisant del counter_object[word].

In [4]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def remove_punctuation(string):
    return re.sub(r'[^a-z0-9 ]', '', string)

def remove_stopwords(word_list):
    return [word for word in word_list if word not in ENGLISH_STOP_WORDS]

ModuleNotFoundError: No module named 'sklearn'

# POUR CÉLINA

Coucou CC, il nous ont donné un file .parquet (juste un autre format de stockage binaire, un peux comme pickle) que normalement j'ai mis dans le dossier Data. Il devrait contenir plusieures informations sur chaque speaker. Tu peux le load comme ça, mais il me semble qu'il faut installer pyarrow (  conda install -c conda-forge pyarrow  ).

Malheureusement il parait qu'ils aient utilisé le QID de Wikidata pour encoder la profession, religion et tout le reste. Probablement une des première choses à faire est de trouver un moyen de mapper les QIDS à des string, que je sais faire en faisant des queries à wikidata mais je me demande s'il n'y a pas un moyen plus simple. Au pire on le fait (que une fois de toute façon).

Il faudrait aussi s'assurer que dans le parquet qu'ils nous donnent il n'y ait pas que peu d'occupations au bol, mais un grand nombre comme celui qu'on voit sur wikidata, car autrement on a le même problème que quand je faisais les queries (que j'ai découvert comment resudre) qui est que pour des gens avec beaucoup de professions on en avait seulement 3 au bol.

Oubliepas que je t'ai dit qu'il y a un problème avec l'antivirus et la lecture de .json. Je ne sais pas pourquoi, mais à chaque fois que tu veux commencer à lire un json tu dois mettre en standby l'anti-virus pendant une minute. C'est assez chiant, mais je n'ai pas trouvé d'autre solution (sinon, le programme ne plante pas, il s'arrete juste et ne dit rien et attend jusqu'à que tu desactives).

Pour le moment c'est tout je crois.

- Cas quand plusieurs qids par quote, lequel on prend ? (je sais pas s'il y a moyen de savoir laquelle est la plus populaire, peut-être celle qui a le plus de liens externes)

In [6]:
import utils

In [26]:
@utils.cache_to_file_pickle("function-groupby_speaker", cache_dir = CACHE_DIR)
def groupby_speaker(input_file_path):
    speakers_dict = {}
    
    with bz2.open(input_file_path, 'rb') as input_file:
        start = time.time()
        
        for i, line in enumerate(input_file):
            line = json.loads(line)
            
            if not line['qids']:
                continue
            
            qids = tuple(line['qids']) if len(line['qids']) > 1 else line['qids'][0]
            
            # if multiple qids given for one quote, take the first one for now
            if qids in speakers_dict:
                speakers_dict[qids]['quote_count'] += 1
                speakers_dict[qids]['speaker']     |= set([line['speaker']])
                # speakers_dict[qids]['num_occurrences'].append(line['numOccurrences'])
                
                speakers_dict[qids]['num_occurrences'].append(len(line['domains']))
                
            else:
                speakers_dict[qids] = {'quote_count': 1, \
                                       'speaker': set([line['speaker']]),
                                       'num_occurrences': [len(line['domains'])]}
                                       # 'num_occurrences': [line['numOccurrences']]}
                
                
            if not i % 1000000:
                print("Read", i, "lines from", input_file_path, 'in', (time.time() - start) / 60, "minutes")
            if i == 1000000:
                break
    return speakers_dict

In [27]:
data_quotes = groupby_speaker(input_file_path = CACHE_FILE_PATH)

Read 0 lines from Cache\processed_data.json.bz2 in 0.00019950469334920247 minutes
Read 1000000 lines from Cache\processed_data.json.bz2 in 0.37840535640716555 minutes


In [28]:
data_quotes_df = pd.DataFrame(data_quotes)

In [30]:
data_quotes_df.T.head(10)

,quote_count,speaker,num_occurrences
Q270316,30,{Jeanne Shaheen},"[2, 2, 1, 2, 1, 1, 2, 2, 3, 2, 1, 1, 1, 1, 2, ..."
Q1253,339,"{Ban Ki Moon, Ban Ki-Moon, Ban Ki-moon, Ban ki...","[2, 1, 99, 1, 2, 1, 2, 91, 7, 1, 3, 2, 12, 1, ..."
Q468374,5,{Sri Sri Ravi Shankar},"[1, 5, 1, 1, 1]"
Q19874690,5,{Jamal Rifi},"[1, 1, 1, 8, 2]"
Q18601741,1,{Richard Burmeister},[1]
Q5271548,9,{Diane Ravitch},"[1, 1, 3, 1, 1, 2, 1, 1, 1]"
"(Q15735939, Q25189328, Q5107375, Q5110828, Q5112832, Q948687)",55,{Chris Matthews},"[1, 1, 3, 1, 3, 1, 2, 2, 1, 1, 1, 1, 2, 1, 3, ..."
Q51797519,5,{Steven Stack},"[1, 1, 1, 1, 1]"
Q2287947,292,"{JORDAN Spieth, Jordan Spieth}","[1, 6, 13, 9, 1, 1, 1, 1, 1, 47, 14, 1, 1, 10,..."
"(Q2376327, Q313381, Q7815037)",368,"{TOM BRADY, Tom Brady}","[1, 1, 1, 1, 7, 5, 1, 2, 1, 10, 1, 2, 2, 1, 4,..."


In [32]:
speaker_data = pd.read_parquet('Data/speaker_attributes.parquet')

In [33]:
speaker_data.id.isna().sum()

0

In [35]:
speaker_data.loc[speaker_data["id"] == 'Q15735939']

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
7078648,[Christopher Douglas Matthews],[+1989-10-06T00:00:00Z],[Q30],[Q6581097],1317867389,[Q49085],None,"[Q19204627, Q19841381]",None,None,Q15735939,Chris Matthews,None,item,None


# POUR ANDREA
Bonjour Andrea, ça va? Oui très bien, merci. J'essaye de poser une base pour commencer le Milestone 2 du projet. Et toi? Moi aussi, drôle ça. Bon, à toute. Bon travail, à toute.

Il faut rerun la lecture du dataset avec la remotion de ponctuation.

Je suppose que tu vas faire la partie de correler les dates à des événements, et si t'as envie d'essayer d'extraire la variance. Par contre pour le moment je ne sais pas trop comment ça colle avec le reste de l'analyse. Dans le sens que la data story va parler de quoi concernant les dates?

# POUR MATTIA
Buongiorno Mattia, je suppose que tu vas faire la partie de regarder les newspapers et correler avec les speakers et, si on arrive, l'argument de la quote. Pour toi je crois que juste load le processed_data.json.bz2 et garder que les clés 'domains' et 'speaker' devrait le faire. Il faudra sûrement se coordonner avec Célina et Kaou pour voir justement comment correler les trucs. Pour le moment, je t'avoue que comme pour ma partie, je n'ai pas une idéé précide de comment ceci va coller dans une data story coherente. Faudra que Kaou et Célina avancent rapidement pour que tu puisse commencer à repliquer leur travail mais pour différents newspapers. Ou je ne sais pas. Faudra juste pas qu'on reste bloqués si on attend quelqu'un d'autre.